In [1]:
import requests
from bs4 import BeautifulSoup
import re as rere
import time
import pandas as pd

In [2]:
re = requests.get('https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=167638&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1')
soup = BeautifulSoup(re.text, 'html.parser')

#### 총 몇 건인지 확인하고 페이지 수 맞추기

In [3]:
howmany = soup.find('strong',{'class':'total'}).findAll('em')[1].get_text()
print(howmany)
#10개가 한 페이지에 존재하므로, 919개의 페이지를 크롤링한다.

9,294


In [4]:
suburl = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=167638&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page='
url = []
for i in range(1,930):
    url.append(suburl+str(i))

ID = []
date = []
score = []
text = []

In [5]:
for link in url:
    re = requests.get(link)
    soup = BeautifulSoup(re.text, 'html.parser')
    source = soup.findAll('div',{'class':'score_reple'})

    ##ID
    #닉네임 / 관람객 태그 제거 후, 아이디만 남기기
    for i in range(10):
        if len(source[i].findAll('span'))==2:
            x = source[i].findAll('span')[1].get_text()
        else:
            x = source[i].find('span').get_text()
        m = rere.search('[a-zA-Z0-9_-]{4}[\*]{4}',x).group()
        ID.append(m)
    ##날짜 / 시간
    source = soup.findAll('div',{'class':'score_reple'})
    for k in range(0,10):
        d = source[k].findAll('em')
        date.append(d[1].get_text())

    ##평점 리스트
    star = soup.findAll('div',{'class':'star_score'})
    for i in range(1,11):
        u = star[i].get_text(); u = rere.sub('\n','',u);
        score.append(u)
    #score = score[1:]
    #score

    ##리뷰 리스트
    for i in range(0,10):
        t = source[i].find('p')
        text.append(t.get_text())

In [ ]:
rst = {'ID': ID,
        'score': score,
        'date' : date,
        'review': text}
data = pd.DataFrame(rst)

### 예시 텍스트 파일

##### 처음 10개 살펴보기

In [7]:
data[:10]

,ID,score,date,review
0,warm****,10,2018.10.31 10:11,촬영 장소? 스케일? 다른 거 다 필요 없다. 최고의 배우들이 자그마한 식탁 위에서...
1,konm****,10,2018.10.31 09:03,관람객사람의 본심은 월식과 같아서 언젠가 드러나는 법
2,yhoc****,8,2018.10.31 10:02,다른 의미로 공포 영화.
3,star****,10,2018.10.31 13:18,저게우리였으면 아무한테도연락안와서 밥만먹고끝났을듯 ㅜ ㅜ
4,norr****,10,2018.10.31 09:40,올해 최고의 블랙코미디. 유해진의 연기는 믿고 봅니다.
5,more****,10,2018.10.31 10:09,관람객간만에 극장에서 다 같이 웃는 경험 한거 같아요 ㅋㅋㅋㅋ 웃음 포인트도 많고 ...
6,jssh****,10,2018.10.31 15:37,57세 키티는 최고!! 간만에 배아프게 웃으며 봤습니다 ^^
7,gch7****,10,2018.10.31 14:57,윤경호라는 배우의 대발견.. 큰 배우들사에서도 전혀 꿀리는거없이 존재감 제대로 사네
8,olzl****,10,2018.10.31 18:56,서로를 잘 모르기에 함께 살 수 있는 우리들
9,tkek****,10,2018.10.31 10:58,요즘 망해가는 한국영화들중에 젤만족스러웠다 막장같은 스토리임에도 깔끔한연출과 배우들...


##### 마지막 10개 살펴보기

In [9]:
data[9280:]

,ID,score,date,review
9280,zzzj****,1,2018.10.31 14:05,와 롯데.. 알바생들 진짜 많이쓴다. 그래서 1점 주고싶어지게 만드네.. 진짜 마케...
9281,happ****,2,2018.11.03 03:41,장소 이동없이 한신으로 찍은 최저 제작비인듯결국..전부 상상이고 ㅡㅡ중간중간 웃음...
9282,dant****,1,2018.10.31 10:06,올해 최악의 영화네요 별점 1점도 아까움
9283,min4****,2,2018.11.04 15:54,이거 보면서 나가고싶은 순간이 10번 넘는다.이거 평점 5점 이상 준 사람 머지 어...
9284,kmh0****,1,2018.11.03 00:22,알바들 때문에 1점 놓고 감 물론 영화는 안봤고 볼 생각도 없음
9285,pari****,1,2018.11.06 00:43,여혐+퀴어비하 끔찍파티
9286,pjb6****,6,2018.11.01 13:55,마지막컷은 이사람들이 게임을하지않았을시 전개과정을 나타냄 10소름인건 이서진이랑 조...
9287,cu_9****,1,2018.11.01 23:16,빵점은 못주나요? 여혐이주류 그리고 첨가된 동성애혐 진짜 여자 가슴이야기가 내용의 ...
9288,poma****,4,2018.10.31 18:43,평점들 믿지 마세요.. 어디가 고오급 드립인지도 모르겠고 문자 올때마다 하나같이 개...
9289,1phi****,2,2018.10.31 10:15,보다초반에짜증나구지루해서 뛰쳐나옴;; 조나재미없구지들밥처먹구수다떠는걸왜 구경해줘야하...


#### xlsx로 저장

In [12]:
data[:10].to_excel("example.xlsx")

#### txt로 저장

In [21]:
text[:10]

['촬영 장소? 스케일? 다른 거 다 필요 없다. 최고의 배우들이 자그마한 식탁 위에서 최상의 요리를 만들어냈다. 어떠한 양념, 조미료도 필요하지 않았다.  ',
 '관람객사람의 본심은 월식과 같아서 언젠가 드러나는 법  ',
 '다른 의미로 공포 영화.  ',
 '저게우리였으면  아무한테도연락안와서 밥만먹고끝났을듯 ㅜ ㅜ  ',
 '올해  최고의 블랙코미디. 유해진의 연기는 믿고 봅니다.  ',
 '관람객간만에 극장에서 다 같이 웃는 경험 한거 같아요 ㅋㅋㅋㅋ 웃음 포인트도 많고 소름돋는 포인트도 있어서 재밌었어요 ㅋㅋㅋ 핸드폰이 누구나 쓰는 물건이라서 더 공감가는 것 같습니당 추천해요  ',
 '57세 키티는 최고!! 간만에 배아프게 웃으며 봤습니다 ^^  ',
 '윤경호라는 배우의 대발견.. 큰 배우들사에서도 전혀 꿀리는거없이 존재감 제대로 사네  ',
 '서로를 잘 모르기에 함께 살 수 있는 우리들  ',
 '요즘 망해가는 한국영화들중에 젤만족스러웠다 막장같은 스토리임에도 깔끔한연출과 배우들의 미친 연기력땜에 쫄리면서도 계속웃겨서쉴틈을 안줌 ㅋㅋㅋ  ']

In [24]:
f = open('assign01.txt','w')
te = text[:10]
for i in te:
    let = "%s \n" %i
    f.write(let)
f.close()